In [25]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lorry\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

Implementazione TFIDF

In [19]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords as sw
import string
import re

class LemmaTokenizer(object):
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
    
    def __call__(self, document):
        lemmas = []
        regex = re.compile("[0-9]+")
        
        for t in word_tokenize(document):
            t = t.strip()
            lemma = self.lemmatizer.lemmatize(t)
            if lemma not in string.punctuation and 3 < len(lemma) < 16 and not regex.match(lemma):
                lemmas.append(lemma)
        return lemmas

Trovo tutte le directory dei documents

In [23]:
rel_path = "data_sets/T-newsgroups/"
documents = [os.path.join(rel_path, d) for d in sorted(os.listdir(rel_path), key=int)]

['data_sets/T-newsgroups/0',
 'data_sets/T-newsgroups/1',
 'data_sets/T-newsgroups/2',
 'data_sets/T-newsgroups/3',
 'data_sets/T-newsgroups/4',
 'data_sets/T-newsgroups/5',
 'data_sets/T-newsgroups/6',
 'data_sets/T-newsgroups/7',
 'data_sets/T-newsgroups/8',
 'data_sets/T-newsgroups/9',
 'data_sets/T-newsgroups/10',
 'data_sets/T-newsgroups/11',
 'data_sets/T-newsgroups/12',
 'data_sets/T-newsgroups/13',
 'data_sets/T-newsgroups/14',
 'data_sets/T-newsgroups/15',
 'data_sets/T-newsgroups/16',
 'data_sets/T-newsgroups/17',
 'data_sets/T-newsgroups/18',
 'data_sets/T-newsgroups/19',
 'data_sets/T-newsgroups/20',
 'data_sets/T-newsgroups/21',
 'data_sets/T-newsgroups/22',
 'data_sets/T-newsgroups/23',
 'data_sets/T-newsgroups/24',
 'data_sets/T-newsgroups/25',
 'data_sets/T-newsgroups/26',
 'data_sets/T-newsgroups/27',
 'data_sets/T-newsgroups/28',
 'data_sets/T-newsgroups/29',
 'data_sets/T-newsgroups/30',
 'data_sets/T-newsgroups/31',
 'data_sets/T-newsgroups/32',
 'data_sets/T-newsgr

Tokenizzo tutte le parole dei documenti tramite il LemmaTokenizer eliminando le stopwords

In [30]:
stopwords = sw.words('english') + ['could', 'might', 'must', 'need', 'would']
lemmaTokenizer = LemmaTokenizer()
vectorizer = TfidfVectorizer(input="filename",tokenizer=lemmaTokenizer, stop_words=stopwords)
tfidf_X = vectorizer.fit_transform(documents)
tfidf_X

<4000x40684 sparse matrix of type '<class 'numpy.float64'>'
	with 357446 stored elements in Compressed Sparse Row format>